In [2]:
import pandas as pd
import tldextract
from urllib.parse import urlparse

# NewsGuard

In [3]:
newsguard_df = pd.read_csv("../data/raw_data/newsguard_20200519.csv", usecols=["Domain"])

In [4]:
newsguard_df.rename(columns={"Domain": "domain"}, inplace=True)

In [5]:
newsguard_df['domain'] = newsguard_df.domain.str.lower()

In [6]:
newsguard_df[newsguard_df.domain.str.contains("/")]

,domain


In [7]:
newsguard_df['suffix'] = newsguard_df.domain.apply(extract_suffix)

NameError: name 'extract_suffix' is not defined

In [35]:
newsguard_df.suffix.value_counts().head(20)

suffix
com      2893
it        458
de        348
fr        300
news      253
org       231
net       121
co.uk     114
site       63
pro        36
tv         33
info       29
us         23
co         19
at         14
eu         12
be         12
xyz        10
ca         10
gov         9
Name: count, dtype: int64

In [10]:
newsguard_df.query("domain == 'valleytimes'")

,domain


In [41]:
newsguard_df.query("suffix == 'edu'")

,domain,suffix
988,brookings.edu,edu
2315,chop.edu,edu
2357,health.harvard.edu,edu


In [44]:
newsguard_df.domain.value_counts().head(20)

domain
vice.com                  5
msn.com                   4
wikipedia.org             4
youtube.com               4
afp.com                   3
blastingnews.com          3
yetiblog.org              2
dna.fr                    2
santepubliquefrance.fr    2
nicematin.com             2
europe1.fr                2
maz-online.de             2
deutschlandfunk.de        2
20minutes.fr              2
policeetrealites.com      2
pisatoday.it              2
rfi.fr                    2
mediapart.fr              2
torinotoday.it            2
topsante.com              2
Name: count, dtype: int64

# MBFC

In [11]:
mbfc_df = pd.read_csv("../data/raw_data/mbfc_ratings.csv", usecols=['source', 'domain'])

In [12]:
mbfc_df.dropna(inplace=True)

In [13]:
mbfc_df.query("domain == 'facebook.com'")

,source,domain
234,https://www.facebook.com/news,facebook.com
2847,https://www.facebook.com/WOMENSRIGHTSNEWS/,facebook.com


In [14]:
mbfc_df.query("domain == 'valleytimes'")

,source,domain


In [8]:
def extract_domain(url):
    """
    Extract the domain from the URL.

    Returns:
        str: The domain (e.g. 'example.com')
    """
    extracted = tldextract.extract(url)
    domain = f"{extracted.domain}.{extracted.suffix}"
    return domain

In [11]:
extract_domain("news.facebook.com")

'facebook.com'

In [25]:
extracted = tldextract.extract('https://www.facebook.com/news')

In [26]:
extracted.domain

'facebook'

In [27]:
extracted.subdomain

'www'

In [28]:
extracted.suffix

'com'

In [9]:
def extract_suffix(url):
    extracted = tldextract.extract(url)
    suffix = extracted.suffix
    return suffix

In [32]:
parsed_url = urlparse('https://www.facebook.com/news')
parsed_url = urlparse('https://www.facebook.com')

In [78]:
def has_path(url):
    parsed_url = urlparse(url)
    path = parsed_url.path.replace("/", "")
    return path != ""

In [45]:
list_of_domains_to_remove = set(
    [
        "youtube.com",
        "twitter.com",
        "instagram.com",
        "tiktok.com",
        "facebook.com",
        "linkedin.com",
        "pinterest.com",
        "reddit.com",
        "tumblr.com",
        "flickr.com",
        "vimeo.com",
        "wikipedia.org",
    ]
)

In [46]:
mbfc_df[mbfc_df.domain.isin(list_of_domains_to_remove)]

,source,domain
234,https://www.facebook.com/news,facebook.com
2385,https://www.youtube.com/nextnewsnetwork,youtube.com
2847,https://www.facebook.com/WOMENSRIGHTSNEWS/,facebook.com
4004,https://www.wikipedia.org/,wikipedia.org


In [79]:
mbfc_df['has_path'] = mbfc_df['source'].apply(has_path)

In [91]:
mbfc_df['suffix'] = mbfc_df.domain.apply(extract_suffix)

In [94]:
mbfc_df.suffix.value_counts().head(20)

suffix
com       3255
org        642
net         90
news        57
ca          38
co.uk       38
us          24
tv          24
com.au      22
gov         19
co          12
info        11
de           8
edu          8
fr           7
it           7
in           7
nl           6
co.za        6
se           5
Name: count, dtype: int64

In [98]:
mbfc_df.query("suffix == 'edu'")

,source,domain,suffix
104,https://www.brookings.edu/,brookings.edu,edu
204,http://earth.columbia.edu/,columbia.edu,edu
758,https://yaleglobal.yale.edu,yale.edu,edu
1349,https://imprimis.hillsdale.edu,hillsdale.edu,edu
2865,https://allianceforscience.cornell.edu,cornell.edu,edu
2988,https://scripps.ucsd.edu/news,ucsd.edu,edu
3345,https://news.harvard.edu/,harvard.edu,edu
3793,http://outlook.monmouth.edu/,outlook.monmouth.edu,edu


In [81]:
temp_df = mbfc_df.query("has_path == True")

In [83]:
temp_df

,source,domain,has_path
3,https://www.whitehouse.gov/1600daily,whitehouse.gov,True
18,https://www.messagemedia.co/aitkin/,messagemedia.co,True
28,https://www.simcoe.com/alliston-on/,simcoe.com,True
29,https://www.northbaynipissing.com/almaguinhigh...,northbaynipissing.com,True
36,https://www.amnesty.org/en/,amnesty.org,True
...,...,...,...
4230,joebiden.com/,joebiden.com,True
4249,https://meduza.io/en,meduza.io,True
4329,https://www.worldsocialism.org/spgb/,worldsocialism.org,True
4364,https://mavenroundtable.io/theintellectualist/,mavenroundtable.io,True


In [84]:
mbfc_df.query("domain == 'whitehouse.gov'")

,source,domain,has_path
3,https://www.whitehouse.gov/1600daily,whitehouse.gov,True
738,https://www.whitehouse.gov,whitehouse.gov,False


In [72]:
parsed_url = urlparse("https://www.100milefreepress.net/news")

In [82]:
temp_df.query("domain == 'facebook.com'")

,source,domain,has_path
234,https://www.facebook.com/news,facebook.com,True
2847,https://www.facebook.com/WOMENSRIGHTSNEWS/,facebook.com,True


In [74]:
parsed_url.path

'/news'

In [41]:
for source in mbfc_df.source:
    try:
        has_path(source)
    except:
        print(source)

nan
nan


In [36]:
not parsed_url.path

True

In [22]:
mbfc_df

,page,mbfclink,bias_rating,factual_reporting,country,media_type,trafficpopularity,credibility_rating,source,domain,leaning,press_freedom_rank
0,100 Mile Free Press,https://mediabiasfactcheck.com/100-mile-free-p...,LEFT-CENTER,HIGH,Canada,Newspaper,Minimal Traffic,HIGH CREDIBILITY,https://www.100milefreepress.net/,100milefreepress.net,leftcenter,NaN
1,604 Now,https://mediabiasfactcheck.com/604-now/,LEFT-CENTER,MOSTLY FACTUAL,Canada,Newspaper,Medium Traffic,HIGH CREDIBILITY,https://604now.com/,604now.com,leftcenter,NaN
2,972 Magazine,https://mediabiasfactcheck.com/972-magazine/,LEFT-CENTER,HIGH,USA,Website,Minimal Traffic,HIGH CREDIBILITY,https://972mag.com/,972mag.com,leftcenter,NaN
3,1600 Daily,https://mediabiasfactcheck.com/1600-daily/,NaN,MOSTLY FACTUAL,USA,NaN,NaN,NaN,https://www.whitehouse.gov/1600daily,whitehouse.gov,leftcenter,NaN
4,ABC News,https://mediabiasfactcheck.com/abc-news/,LEFT-CENTER,HIGH,USA,TV Station,High Traffic,HIGH CREDIBILITY,https://abcnews.go.com,abcnews.go.com,leftcenter,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4416,World Socialist Web Site,https://mediabiasfactcheck.com/world-socialist...,LEFT,MOSTLY FACTUAL,USA (44/180 Press Freedom),Website,Medium Traffic,HIGH CREDIBILITY,https://www.wsws.org,wsws.org,left,NaN
4417,Yes Magazine,https://mediabiasfactcheck.com/yes-magazine/,LEFT,HIGH,USA,Magazine,Medium Traffic,HIGH CREDIBILITY,https://www.yesmagazine.org,yesmagazine.org,left,NaN
4418,Young Turks,https://mediabiasfactcheck.com/the-young-turks/,LEFT,MIXED,USA,TV Station,High Traffic,MEDIUM CREDIBILITY,https://tyt.com/,tyt.com,left,NaN
4419,Your Black World,https://mediabiasfactcheck.com/your-black-world/,LEFT,MIXED,USA,NaN,NaN,NaN,http://yourblackworld.net,yourblackworld.net,left,NaN


In [6]:
len(mbfc_df)

4421

In [7]:
mbfc_df.dropna(inplace=True)

In [9]:
len(mbfc_df)

4416

In [10]:
mbfc_df['domain'] = mbfc_df.domain.str.lower()

In [12]:
mbfc_df.drop_duplicates(inplace=True)

In [13]:
len(mbfc_df)

4275

In [17]:
mbfc_df[mbfc_df.domain.str.contains("/")]

,domain


In [16]:
mbfc_df['domain'] = mbfc_df.domain.str.replace("/", "")

In [18]:
mbfc_df

,domain
0,100milefreepress.net
1,604now.com
2,972mag.com
3,whitehouse.gov
4,abcnews.go.com
...,...
4415,workers.org
4416,wsws.org
4417,yesmagazine.org
4418,tyt.com


# Robterson

In [15]:
robertson_df = pd.read_csv("../data/raw_data/robertson_domains.tsv", sep='\t', usecols=['domain', 'news_is_news'])

In [16]:
robertson_df = robertson_df[robertson_df.news_is_news.notna()].copy()

In [17]:
robertson_df['is_news'] = robertson_df.news_is_news.astype(int)

In [18]:
robertson_df = robertson_df[robertson_df.is_news == 1][['domain']].copy()

In [19]:
robertson_df['domain'] = robertson_df['domain'].str.lower()

In [71]:
robertson_df[robertson_df.domain.str.contains("/")]

,domain


In [20]:
robertson_df.query("domain == 'valleytimes'")

,domain


In [72]:
robertson_df['suffix'] = robertson_df.domain.apply(extract_suffix)

In [73]:
robertson_df.suffix.value_counts()

suffix
com        6662
org         348
net         272
edu          72
tv           65
us           27
co           22
info         14
news         12
co.uk        11
gov           9
biz           7
me            4
com.co        4
ca            4
com.mx        3
ms            3
fr            2
coop          2
to            2
today         2
ly            2
is            2
cc            2
tk            2
de            2
ng            1
ms.us         1
ir            1
fm            1
io            1
mn            1
co.za         1
media         1
site          1
es            1
wy.us         1
nj.us         1
travel        1
id.us         1
rs            1
ag            1
ee            1
website       1
link          1
co.il         1
in            1
co.jp         1
sc            1
world         1
buzz          1
Name: count, dtype: int64

In [78]:
robertson_df.domain.value_counts()

domain
1.001.free.fr                 1
rockfordsquire.com            1
rollingthunderexpress.com     1
rollingstone.com              1
rollcall.com                  1
                             ..
hutchnews.com                 1
huskers.com                   1
hurherald.com                 1
huntnewsnu.com                1
woundedamericanwarrior.com    1
Name: count, Length: 7582, dtype: int64

In [79]:
len(robertson_df)

7582

In [76]:
robertson_df.query("suffix == 'ly'")

,domain,suffix
13058,personaldefenseworld.com.via.snip.ly,ly
18821,usat.ly,ly


In [49]:
robertson_df.news_is_news.value_counts()

news_is_news
0.0    13207
1.0     7582
Name: count, dtype: int64

In [52]:
robertson_df[robertson_df.news_is_news != 1.0]

,domain,news_fakenews_proj,news_googlenews,news_manual,news_tvnews_data,news_bakshy,news_yin,news_n,news_is_news,bias_score,bias_rounded_n_shares,bias_rounded_n_accts,bias_fb_score,bias_mturk_score,bias_budak_score,bias_allsides_score,bias_allsides_score_community,bias_pew_score,fn_color,fn_is_fake
0,007.com,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0948,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100isnow.com,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,-0.7228,100.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100photos.time.com,NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1011now.com,NaN,1.0,NaN,NaN,NaN,NaN,0.0,0.0,0.5387,1700.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,101greatgoals.com,NaN,1.0,NaN,NaN,NaN,NaN,0.0,0.0,-0.1426,300.0,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31904,zyn.ga,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0883,1000.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31905,zynga.tm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.1284,5500.0,1100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31906,hillaryspeeches.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
31907,sli.mg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [57]:
robertson_df.query("domain == 'whitehouse.gov'")

,domain,news_fakenews_proj,news_googlenews,news_manual,news_tvnews_data,news_bakshy,news_yin,news_n,news_is_news,bias_score,bias_rounded_n_shares,bias_rounded_n_accts,bias_fb_score,bias_mturk_score,bias_budak_score,bias_allsides_score,bias_allsides_score_community,bias_pew_score,fn_color,fn_is_fake
19839,whitehouse.gov,0.0,NaN,NaN,NaN,1.0,NaN,1.0,1.0,-0.2283,33200.0,21500.0,-0.5325,0.25,NaN,0.5,1.0,NaN,NaN,NaN


In [ ]:
rokkk

# Hohenberg

In [21]:
hohenberg_df = pd.read_csv("../data/raw_data/hohenberg.csv")

In [22]:
hohenberg_df[hohenberg_df.domain.isna()]

,domain,ideology,type


In [23]:
hohenberg_df[hohenberg_df.domain.str.contains("/")]

,domain,ideology,type
159,aol.com/news,NaN,NaN
357,bing.com/news,NaN,NaN
2810,msn.com/en-us/news,0.1119,NaN
5362,yahoo.com/news,0.0848,national


In [24]:
hohenberg_df.query("domain == 'valleytimes'")

,domain,ideology,type


In [89]:
def has_path(url):
    """
    Check if the URL has a path component.

    Returns:
        bool: True if URL has a non-empty path, False otherwise
    """
    parsed_url = urlparse(url)
    path = parsed_url.path.replace("/", "")
    print(path)
    return path != ""

In [90]:
has_path("aol.com/news")

aol.comnews


True

In [91]:
has_path("aol.com")

aol.com


True

In [93]:
parsed_url = urlparse("aol.com")

In [94]:
parsed_url

ParseResult(scheme='', netloc='', path='aol.com', params='', query='', fragment='')

# Le Quere

In [25]:
le_quere_df = pd.read_csv("../data/raw_data/le_quere.csv", usecols=['WebsiteUrl'])

In [26]:
le_quere_df.rename(columns={"WebsiteUrl": "domain"}, inplace=True)

In [27]:
le_quere_df = le_quere_df[le_quere_df.domain.notna()].copy()

In [28]:
le_quere_df[le_quere_df.domain.str.contains("/")]

,domain
23,peninsulaclarion.com/dispatch
35,seldovia.com/seldovia-gazette
63,gulfcoastnewstoday.com/the_baldwin_times
64,linktr.ee/abouttownmag
68,al.com/birmingham
...,...
10230,patch.com/pennsylvania/yardley
10232,newsbreak.com/pennsylvania/york
10236,newsbreak.com/california/yucaipa
10237,newsbreak.com/arizona/yuma


In [110]:
le_quere_df = le_quere_df[~le_quere_df.domain.str.contains("/")].copy()

In [112]:
le_quere_df['domain'] = le_quere_df.domain.str.lower()

In [116]:
le_quere_df['suffix'] = le_quere_df.domain.apply(extract_suffix)

In [118]:
le_quere_df.suffix.value_counts()

suffix
com              7619
net               364
org               117
news               22
us                 21
info               16
ca                 10
co.uk               8
cz                  8
biz                 6
me                  5
tv                  5
co                  4
se                  4
ms                  4
in                  3
fi                  3
nl                  2
ie                  2
com.au              2
gov                 2
com.do              2
online              2
cv                  2
edu                 2
eu                  2
plus                1
com.tr              1
no                  1
buzz                1
co.zw               1
fr                  1
coop                1
pl                  1
fm                  1
cc                  1
today               1
la                  1
org.tw              1
ad                  1
life                1
ma                  1
it                  1
mn                  1
                    1
co.

In [121]:
le_quere_df.query("suffix == 'edu'")

,domain,suffix
4248,thesouthend.wayne.edu,edu
5846,paw.princeton.edu,edu


In [29]:
le_quere_df.query("domain == 'valleytimes'")

,domain


# Fisher

In [30]:
fisher_df = pd.read_csv("../data/raw_data/fisher.csv", usecols=['domain'])

In [31]:
fisher_df[fisher_df.domain.isna()]

,domain


In [32]:
fisher_df[fisher_df.domain.str.contains("/")]

,domain
1164,cleveland.cbslocal.com/


In [ ]:
fisher_df[fisher_df.domain.str.contains("/")]

In [33]:
fisher_df.query("domain == 'valleytimes'")

,domain


# Horne

In [34]:
horne_raw_df = pd.read_csv("../data/raw_data/horne.csv", usecols=['sourcedomain_id'])

In [35]:
horne_domains = []
for index, row in horne_raw_df.iterrows():
    horne_domains.append(row["sourcedomain_id"].split("-")[1])

In [36]:
horne_df = pd.DataFrame(horne_domains, columns=["domain"])
horne_df.drop_duplicates(inplace=True)

In [37]:
horne_df[horne_df.domain.isna()]

,domain


In [38]:
horne_df[horne_df.domain.str.contains("/")]

,domain


In [40]:
horne_df.query("domain == 'advocate'")

,domain
27,advocate


# ABYZ

In [193]:
def extract_domain(url):
    """
    Extract the domain from the URL.

    Returns:
        str: The domain (e.g. 'example.com')
    """
    extracted = tldextract.extract(url)
    domain = f"{extracted.subdomain}.{extracted.domain}.{extracted.suffix}"
    return domain

In [153]:
abyz_df = pd.read_csv("../data/raw_data/abyz.csv")#, usecols=['domain'])

In [ ]:
abyz_df

In [187]:
temp_abyz_df = abyz_df[abyz_df.domain.str.contains("www")].copy()

In [194]:
temp_abyz_df['clean_domain'] = temp_abyz_df.domain.apply(extract_domain)

In [196]:
temp_abyz_df[temp_abyz_df.clean_domain.str.endswith("gov")]

,scope,url,media_name,media_type,media_focus,language,domain,state_abbr,clean_domain


In [149]:
abyz_df[abyz_df.domain.isna()]

,domain


In [150]:
abyz_df[abyz_df.domain.str.contains("/")]

,domain


In [151]:
abyz_df.domain.value_counts()

domain
smalltownpapers.com    176
wickedlocal.com        112
mysuburbanlife.com      60
bizjournals.com         42
northjersey.com         42
                      ... 
the-papers.com           1
mvdemocrat.com           1
bsudailynews.com         1
munciefreepress.com      1
wyodaily.com             1
Name: count, Length: 10414, dtype: int64

In [154]:
abyz_df.query("domain == 'smalltownpapers.com'")

,scope,url,media_name,media_type,media_focus,language,domain,state_abbr
383,Florala,http://www.smalltownpapers.com/newspapers/news...,Florala News,NP,GI,ENG,smalltownpapers.com,al
433,Linden,http://www.smalltownpapers.com/newspapers/news...,Democrat Reporter,NP,GI,ENG,smalltownpapers.com,al
502,Tuskegee,http://www.smalltownpapers.com/newspapers/news...,Tuskegee News,NP,GI,ENG,smalltownpapers.com,al
504,Vernon,http://www.smalltownpapers.com/newspapers/news...,Lamar Democrat and Sulligent,NP,GI,ENG,smalltownpapers.com,al
589,Ajo,http://www.smalltownpapers.com/newspapers/news...,Ajo Copper News,NP,GI,ENG,smalltownpapers.com,az
...,...,...,...,...,...,...,...,...
11946,Sammamish,http://www.smalltownpapers.com/newspapers/news...,Sammamish Review,NP,GI,ENG,smalltownpapers.com,wa
11998,Seattle (West),http://www.smalltownpapers.com/newspapers/news...,West Seattle Herald White Center,NP,GI,ENG,smalltownpapers.com,wa
12003,Shelton,http://www.smalltownpapers.com/newspapers/news...,Shelton Mason County Journal,NP,GI,ENG,smalltownpapers.com,wa
12065,Woodinville,http://www.smalltownpapers.com/newspapers/news...,Woodinville Weekly,NP,GI,ENG,smalltownpapers.com,wa


# Combined data

In [172]:
combined_df = pd.read_csv("../data/intermediate_files/cleaned_combined.csv")

In [173]:
combined_df.domain.nunique()

20548

In [158]:
combined_df.domain.value_counts().value_counts()

count
1    12224
5     2995
2     2148
3     1033
4     1001
6      743
7      368
8       36
Name: count, dtype: int64

In [177]:
combined_df[combined_df.domain.str.endswith("edu")].dataset.value_counts()

dataset
abyz         103
fisher        83
robertson     72
mbfc           7
newsguard      3
hohenberg      3
le_quere       2
Name: count, dtype: int64

In [181]:
combined_df.query("domain == 'www.cw.ua.edu'")

,domain,dataset
35965,www.cw.ua.edu,abyz


# Tranco list

In [162]:
tranco_df = pd.read_csv("../data/raw_data/tranco_list_20240101.csv", names=['rank', 'domain'])

In [167]:
combine_unique_df = pd.read_csv("../data/intermediate_files/combined_unique.csv")

In [168]:
domain_with_rank_df = combine_unique_df.merge(tranco_df, on='domain')

In [171]:
domain_with_rank_df.sort_values(by="rank", inplace=True)

In [ ]:
domain_with_rank_df.

In [197]:
gov_edu_df = pd.read_csv("../data/raw_data/gov_edu_domains_to_exclude.csv")

In [199]:
gov_edu_df.dtypes

domain     object
suffix     object
exclude      bool
dtype: object

In [200]:
gov_edu_df[gov_edu_df['exclude']]

,domain,suffix,exclude
0,brookings.edu,edu,True
1,chop.edu,edu,True
3,columbia.edu,edu,True
4,yale.edu,edu,True
5,hillsdale.edu,edu,True
...,...,...,...
218,wyo.gov,gov,True
219,chroniclingamerica.loc.gov,gov,True
220,paulding.gov,gov,True
221,collincountytx.gov,gov,True


In [201]:
final_df = pd.read_csv("../data/output/news_domains.csv")

In [205]:
final_df[final_df.domain.str.endswith("gov")]

,domain


In [207]:
final_df[final_df.domain.str.contains("www")]

,domain
2674,qa-www.nbcsports.com
12432,www1.cbn.com
15597,www.alabama.statenews.net
15665,www.cw.ua.edu
15710,www.arizona.statenews.net
...,...
19908,www.fairfax.daily-monitor.com
19924,www.loudoun.daily-monitor.com
19961,www.winchester.daily-monitor.com
19966,www.washington.statenews.net


# Media cloud

In [4]:
media_cloud_df_raw = pd.read_parquet("../data/raw_data/media_cloud_us_sources_20250326.parquet")

In [5]:
media_cloud_df = media_cloud_df_raw[['name']].copy()

In [7]:
media_cloud_df.rename(columns={"name": "domain"}, inplace=True)

In [9]:
media_cloud_df[media_cloud_df.domain.isna()]

,domain


In [11]:
media_cloud_df['domain'] = media_cloud_df['domain'].str.lower()

In [12]:
media_cloud_df[media_cloud_df.domain.str.contains("/")]

,domain


In [13]:
len(media_cloud_df)

27642

In [14]:
media_cloud_df.drop_duplicates(inplace=True)

In [15]:
len(media_cloud_df)

9924

In [16]:
gov_edu_domain_to_exclude_df = pd.read_csv("../data/raw_data/gov_edu_domains_to_exclude.csv")

In [24]:
gov_edu_domains_dup = gov_edu_domain_to_exclude_df[['domain', 'exclude']].dropna().drop_duplicates()

In [27]:
gov_edu_domains_dup.domain.value_counts().value_counts()

count
1    270
Name: count, dtype: int64